![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [0]:
from johnsnowlabs import nlp, finance, viz

🚨 Your Spark-OCR is outdated, installed==4.3.0 but latest version==4.3.1
You can run nlp.install() to update Spark-OCR

##🔎  Pretrained Relation Extraction Models for Finance

Here are the list of pretrained Relation Extraction models:

📜**Relation Extraction Models**

|index|model|
|-----:|:-----|
| 1| [Financial Relation Extraction on Earning Calls (Small)](https://nlp.johnsnowlabs.com/2022/11/28/finre_earning_calls_sm_en.html)  | 
| 2| [Financial Relation Extraction on 10K filings (Small)](https://nlp.johnsnowlabs.com/2022/11/07/finre_financial_small_en.html)  | 
| 3| [Financial Relation Extraction (Tickers)](https://nlp.johnsnowlabs.com/2022/10/15/finre_has_ticker_en.html)  |
| 4| [Financial Relation Extraction (Acquisitions / Subsidiaries)](https://nlp.johnsnowlabs.com/2022/11/08/finre_acquisitions_subsidiaries_md_en.html)  | 
| 5| [Financial Relation Extraction (Work Experience, Medium)](https://nlp.johnsnowlabs.com/2022/11/08/finre_work_experience_md_en.html)  |
| 6| [Financial Relation Extraction (Work Experience, Small)](https://nlp.johnsnowlabs.com/2022/09/28/finre_work_experience_en.html)  | 
| 7| [Financial Relation Extraction (Alias)](https://nlp.johnsnowlabs.com/2022/08/17/finre_org_prod_alias_en_3_2.html)  |
| 8| [Financial Zero-shot Relation Extraction](https://nlp.johnsnowlabs.com/2022/08/22/finre_zero_shot_en_3_2.html)  |

**These components are common for all the pipelines we will use.**

In [0]:
def get_generic_base_pipeline():
  """Common components used in all pipelines"""
  document_assembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  text_splitter = finance.TextSplitter()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")
  
  tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  base_pipeline = nlp.Pipeline(stages=[
      document_assembler,
      text_splitter,
      tokenizer,
      embeddings
  ])

  return base_pipeline
    
generic_base_pipeline = get_generic_base_pipeline()

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

In [0]:
# Text Classifier
def get_text_classification_pipeline(model):
  """This pipeline allows you to use different classification models to understand if an input text is of a specific class or is something else.
  It will be used to check where the first summary page of SEC10K is, where the sections of Acquisitions and Subsidiaries are, or where in the document
  the management roles and experiences are mentioned"""
  document_assembler = nlp.DocumentAssembler() \
       .setInputCol("text") \
       .setOutputCol("document")

  embeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

  classifier = nlp.ClassifierDLModel.pretrained(model, "en", "finance/models")\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("category")

  nlpPipeline = nlp.Pipeline(stages=[
      document_assembler, 
      embeddings,
      classifier])
  
  return nlpPipeline

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  """Shows a Dataframe with the relations extracted by Spark NLP"""
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
        rel.result, 
        rel.metadata['entity1'], 
        rel.metadata['entity1_begin'],
        rel.metadata['entity1_end'],
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['entity2_begin'],
        rel.metadata['entity2_end'],
        rel.metadata['chunk2'], 
        rel.metadata['confidence']
    ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df

##🔎  Zero-shot Relation Extraction to Extract Relations Between Financial Entities

Let's suppose now we want to extract relations between `PROFIT_DECLINE`,  `PROFIT_INCREASE`, `EXPENSE_DECREASE`, `AMOUNT`, `PERCENTAGE` entities. Right now, we don't have a model to do that, but!

That's when Zero-shot RE comes into the game. You can use Zero-shot RE **without training data** and **without any pretrained model** to create your REDL model.


📜At John Snow Labs, we have developed our own annotators based on **Natural Language Inference (NLI)**, to not only carry out Question Answering, but using QA to:
- Retrieve Entities, also known as Zero-shot NER;
- Retrieve Relations, also known as Zero-shot Relation Extraction;

##🔎  A variation of NLI for Zero-shot Relation Extraction
Similarly to Zero-shot NER, Zero-shot RE also works with `H` (hypotheses) and `P` (premises), and the extraction as a positive result is conditioned to the `H` being `entailed` given a `P`.

📜In this case, what we do is:
- We took a prompt in the form of {ENT_1} [some_text] {ENT_2}
- ENT_1 is filled with entities from a previous NER
- ENT_2 too.
- We ask the ZeroShotRE model if, given the whole text, the premise {ENT_1} [some_text] {ENT_2} is entailed.

For example, `ENT_1` is `REVENUE`. `ENT_2` is `PERCENTAGE`. `[some_text]` is `decreased`.

Given a premise `License fees revenue decreased 40 %`, the result of the previous prompt will be `entailed`, returning a positive as a result.

However, `License fees revenue increased 40 %` would not return an entailment, so the relation will not be triggered.

##📌 Example

Firstly, we use the `finner_financial_small` model to extract `PROFIT_DECLINE`,  `PROFIT_INCREASE`, `EXPENSE_DECREASE`, `AMOUNT`, `PERCENTAGE`, entities. After that we define relations between these entities paying attention to the syntax.

📜For example, given the text P,`License fees revenue decreased 40 %, or $ 0.5 million to $ 0.7 million for the year ended December 31, 2020 compared to $ 1.2 million for the year ended December 31, 2019.` we:

- Generate Hypotheses H with the tokens of the text
  - License fees revenue increase 40: `contradiction`
  - License fees revenue decrease 40: `entailment`
  - License fees revenue decrease 1.2 million: `entailment`
  - License fees revenue decrease 0.7 million: `entailment`
  - License fees revenue decrease 0.5 million: `entailment`

- We check all the H towards P to see if they are `entailed`. If so, we return them as relations between the entities.


🚀**!!! Make sure you keep the proper syntax of the relations you want to extract !!!**

In [0]:
ner_model = finance.NerModel.pretrained("finner_financial_small", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

re_model = finance.ZeroShotRelationExtractionModel.pretrained("finre_zero_shot", "en", "finance/models")\
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("relations")\
    .setMultiLabel(False)

re_model.setRelationalCategories({
    "DECREASE": ["{PROFIT_DECLINE} decrease to {AMOUNT}", "{PROFIT_DECLINE} decrease {PERCENTAGE}", "{EXPENSE_DECREASE} decrease {AMOUNT}", "{EXPENSE_DECREASE} decrease {PERCENTAGE}"],
    "INCREASE": ["{PROFIT_INCREASE} increase to {AMOUNT}", "{PROFIT_INCREASE} increase {PERCENTAGE}"],
})

pipeline = nlp.Pipeline(stages =[
                generic_base_pipeline,
                ner_model,
                ner_converter,
                re_model
               ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

light_model = nlp.LightPipeline(model)

finner_financial_small download started this may take some time.
[ | ][ / ][ — ][OK!]
finre_zero_shot download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
sample_text = """License fees revenue decreased 40 %, or $ 0.5 million to $ 0.7 million for the year ended December 31, 2020 compared to $ 1.2 million for the year ended December 31, 2019. Services revenue increased 4 %, or $ 1.1 million, to $ 25.6 million for the year ended December 31, 2020 from $ 24.5 million for the year ended December 31, 2019. Sales and marketing expenses decreased 20 %, or $ 1.5 million, to $ 6.0 million for the year ended December 31, 2020 from $ 7.5 million for the year ended December 31, 2019."""

sample_text


Out[6]: 'License fees revenue decreased 40 %, or $ 0.5 million to $ 0.7 million for the year ended December 31, 2020 compared to $ 1.2 million for the year ended December 31, 2019. Services revenue increased 4 %, or $ 1.1 million, to $ 25.6 million for the year ended December 31, 2020 from $ 24.5 million for the year ended December 31, 2019. Sales and marketing expenses decreased 20 %, or $ 1.5 million, to $ 6.0 million for the year ended December 31, 2020 from $ 7.5 million for the year ended December 31, 2019.'

In [0]:
data = spark.createDataFrame([[sample_text]]).toDF("text")

result = model.transform(data)

###🖨️ Get Results

**NER output**

In [0]:
result.selectExpr("explode(ner_chunk) as ner").show(50, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+
ner |
+---------------------------------------------------------------------------------------------------------------------------------------+
{chunk, 0, 19, License fees revenue, {entity -> PROFIT_DECLINE, sentence -> 0, chunk -> 0, confidence -> 0.56216663}, []} |
{chunk, 31, 32, 40, {entity -> PERCENTAGE, sentence -> 0, chunk -> 1, confidence -> 1.0}, []} |
{chunk, 40, 40, $, {entity -> CURRENCY, sentence -> 0, chunk -> 2, confidence -> 1.0}, []} |
{chunk, 42, 52, 0.5 million, {entity -> AMOUNT, sentence -> 0, chunk -> 3, confidence -> 1.0}, []} |
{chunk, 57, 57, $, {entity -> CURRENCY, sentence -> 0, chunk -> 4, confidence -> 1.0}, []} |
{chunk, 59, 69, 0.7 million, {entity -> AMOUNT, sentence -> 0, chunk -> 5, confidence -> 0.99995}, []} |
{chunk, 90, 106, December 31, 2020, {entity -> FISCAL_YEAR, sentence -> 0, chunk -> 6, confidence -> 0.98075}, []} |
{chunk, 120, 120, $, {entity -> CURRENCY, sentence -> 0, chunk -> 7, confidence -> 1.0}, []} |
{chunk, 122, 132, 1.2 million, {entity -> AMOUNT, sentence -> 0, chunk -> 8, confidence -> 1.0}, []} |
{chunk, 153, 169, December 31, 2019, {entity -> FISCAL_YEAR, sentence -> 0, chunk -> 9, confidence -> 0.945875}, []} |
{chunk, 172, 187, Services revenue, {entity -> PROFIT_INCREASE, sentence -> 1, chunk -> 10, confidence -> 0.83635}, []} |
{chunk, 199, 199, 4, {entity -> PERCENTAGE, sentence -> 1, chunk -> 11, confidence -> 1.0}, []} |
{chunk, 207, 207, $, {entity -> CURRENCY, sentence -> 1, chunk -> 12, confidence -> 1.0}, []} |
{chunk, 209, 219, 1.1 million, {entity -> AMOUNT, sentence -> 1, chunk -> 13, confidence -> 1.0}, []} |
{chunk, 225, 225, $, {entity -> CURRENCY, sentence -> 1, chunk -> 14, confidence -> 1.0}, []} |
{chunk, 227, 238, 25.6 million, {entity -> AMOUNT, sentence -> 1, chunk -> 15, confidence -> 1.0}, []} |
{chunk, 259, 275, December 31, 2020, {entity -> FISCAL_YEAR, sentence -> 1, chunk -> 16, confidence -> 0.9804}, []} |
{chunk, 282, 282, $, {entity -> CURRENCY, sentence -> 1, chunk -> 17, confidence -> 1.0}, []} |
{chunk, 284, 295, 24.5 million, {entity -> AMOUNT, sentence -> 1, chunk -> 18, confidence -> 1.0}, []} |
{chunk, 316, 332, December 31, 2019, {entity -> FISCAL_YEAR, sentence -> 1, chunk -> 19, confidence -> 0.937975}, []} |
{chunk, 335, 362, Sales and marketing expenses, {entity -> EXPENSE_DECREASE, sentence -> 2, chunk -> 20, confidence -> 0.65842503}, []}|
{chunk, 374, 375, 20, {entity -> PERCENTAGE, sentence -> 2, chunk -> 21, confidence -> 1.0}, []} |
{chunk, 383, 383, $, {entity -> CURRENCY, sentence -> 2, chunk -> 22, confidence -> 1.0}, []} |
{chunk, 385, 395, 1.5 million, {entity -> AMOUNT, sentence -> 2, chunk -> 23, confidence -> 1.0}, []} |
{chunk, 401, 401, $, {entity -> CURRENCY, sentence -> 2, chunk -> 24, confidence -> 1.0}, []} |
{chunk, 403, 413, 6.0 million, {entity -> AMOUNT, sentence -> 2, chunk -> 25, confidence -> 1.0}, []} |
{chunk, 434, 450, December 31, 2020, {entity -> FISCAL_YEAR, sentence -> 2, chunk -> 26, confidence -> 0.97915}, []} |
{chunk, 457, 457, $, {entity -> CURRENCY, sentence -> 2, chunk -> 27, confidence -> 1.0}, []} |
{chunk, 459, 469, 7.5 million, {entity -> AMOUNT, sentence -> 2, chunk -> 28, confidence -> 1.0}, []} |
{chunk, 490, 506, December 31, 2019, {entity -> FISCAL_YEAR, sentence -> 2, chunk -> 29, confidence -> 0.94195}, []} |
+---------------------------------------------------------------------------------------------------------------------------------------+

**Relations output**

In [0]:
light_result = light_model.fullAnnotate(sample_text)

rel_df = get_relations_df(light_result)

rel_df[rel_df["relation"] != "no_rel"]

Out[9]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 INCREASE 
 PROFIT_INCREASE 
 172 
 187 
 Services revenue 
 AMOUNT 
 227 
 238 
 25.6 million 
 0.97882193 
 
 
 1 
 DECREASE 
 EXPENSE_DECREASE 
 335 
 362 
 Sales and marketing expenses 
 PERCENTAGE 
 374 
 375 
 20 
 0.9928862 
 
 
 2 
 DECREASE 
 EXPENSE_DECREASE 
 335 
 362 
 Sales and marketing expenses 
 AMOUNT 
 385 
 395 
 1.5 million 
 0.9894141 
 
 
 3 
 INCREASE 
 PROFIT_INCREASE 
 172 
 187 
 Services revenue 
 AMOUNT 
 209 
 219 
 1.1 million 
 0.9759809 
 
 
 4 
 DECREASE 
 EXPENSE_DECREASE 
 335 
 362 
 Sales and marketing expenses 
 AMOUNT 
 459 
 469 
 7.5 million 
 0.9819981 
 
 
 5 
 DECREASE 
 EXPENSE_DECREASE 
 335 
 362 
 Sales and marketing expenses 
 AMOUNT 
 403 
 413 
 6.0 million 
 0.9839978 
 
 
 6 
 INCREASE 
 PROFIT_INCREASE 
 172 
 187 
 Services revenue 
 AMOUNT 
 284 
 295 
 24.5 million 
 0.9391607 
 
 
 7 
 DECREASE 
 PROFIT_DECLINE 
 0 
 19 
 License fees revenue 
 PERCENTAGE 
 31 
 32 
 40 
 0.9931541 
 
 
 8 
 DECREASE 
 PROFIT_DECLINE 
 0 
 19 
 License fees revenue 
 AMOUNT 
 122 
 132 
 1.2 million 
 0.7389867 
 
 
 9 
 DECREASE 
 PROFIT_DECLINE 
 0 
 19 
 License fees revenue 
 AMOUNT 
 59 
 69 
 0.7 million 
 0.9894014 
 
 
 10 
 INCREASE 
 PROFIT_INCREASE 
 172 
 187 
 Services revenue 
 PERCENTAGE 
 199 
 199 
 4 
 0.97684693 
 
 
 11 
 DECREASE 
 PROFIT_DECLINE 
 0 
 19 
 License fees revenue 
 AMOUNT 
 42 
 52 
 0.5 million 
 0.9900114

In [0]:
# relations output
result.selectExpr("explode(relations) as relation").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
relation |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
{category, 1323, 1539, INCREASE, {entity1_begin -> 172, relation -> INCREASE, hypothesis -> Services revenue increase to 25.6 million, confidence -> 0.97882193, nli_prediction -> entail, entity1 -> PROFIT_INCREASE, syntactic_distance -> undefined, chunk2 -> 25.6 million, entity2_end -> 238, entity1_end -> 187, entity2_begin -> 227, entity2 -> AMOUNT, chunk1 -> Services revenue, sentence -> 1}, []} |
{category, 1757, 1983, DECREASE, {entity1_begin -> 335, relation -> DECREASE, hypothesis -> Sales and marketing expenses decrease 20, confidence -> 0.9928862, nli_prediction -> entail, entity1 -> EXPENSE_DECREASE, syntactic_distance -> undefined, chunk2 -> 20, entity2_end -> 375, entity1_end -> 362, entity2_begin -> 374, entity2 -> PERCENTAGE, chunk1 -> Sales and marketing expenses, sentence -> 2}, []} |
{category, 1984, 2219, DECREASE, {entity1_begin -> 335, relation -> DECREASE, hypothesis -> Sales and marketing expenses decrease 1.5 million, confidence -> 0.9894141, nli_prediction -> entail, entity1 -> EXPENSE_DECREASE, syntactic_distance -> undefined, chunk2 -> 1.5 million, entity2_end -> 395, entity1_end -> 362, entity2_begin -> 385, entity2 -> AMOUNT, chunk1 -> Sales and marketing expenses, sentence -> 2}, []}|
{category, 1107, 1322, INCREASE, {entity1_begin -> 172, relation -> INCREASE, hypothesis -> Services revenue increase to 1.1 million, confidence -> 0.97598094, nli_prediction -> entail, entity1 -> PROFIT_INCREASE, syntactic_distance -> undefined, chunk2 -> 1.1 million, entity2_end -> 219, entity1_end -> 187, entity2_begin -> 209, entity2 -> AMOUNT, chunk1 -> Services revenue, sentence -> 1}, []} |
{category, 2456, 2691, DECREASE, {entity1_begin -> 335, relation -> DECREASE, hypothesis -> Sales and marketing expenses decrease 7.5 million, confidence -> 0.9819981, nli_prediction -> entail, entity1 -> EXPENSE_DECREASE, syntactic_distance -> undefined, chunk2 -> 7.5 million, entity2_end -> 469, entity1_end -> 362, entity2_begin -> 459, entity2 -> AMOUNT, chunk1 -> Sales and marketing expenses, sentence -> 2}, []}|
{category, 2220, 2455, DECREASE, {entity1_begin -> 335, relation -> DECREASE, hypothesis -> Sales and marketing expenses decrease 6.0 million, confidence -> 0.9839978, nli_prediction -> entail, entity1 -> EXPENSE_DECREASE, syntactic_distance -> undefined, chunk2 -> 6.0 million, entity2_end -> 413, entity1_end -> 362, entity2_begin -> 403, entity2 -> AMOUNT, chunk1 -> Sales and marketing expenses, sentence -> 2}, []}|
{category, 1540, 1756, INCREASE, {entity1_begin -> 172, relation -> INCREASE, hypothesis -> Services revenue increase to 24.5 million, confidence -> 0.9391607, nli_prediction -> entail, entity1 -> PROFIT_INCREASE, syntactic_distance -> undefined, chunk2 -> 24.5 million, entity2_end -> 295, entity1_end -> 187, entity2_begin -> 284, entity2 -> AMOUNT, chunk1 -> Services revenue, sentence -> 1}, []} |
{category, 0, 216, DECREASE, {entity1_begin -> 0, relation -> DECREASE, hypothesis -> License fees revenue decrease 40, confidence -> 0.9931541, nli_prediction -> entail, entity1 -> PROFIT_DECLINE, syntactic_distance -> undefined, chunk2 -> 40, entity2_end -> 32, enti

###🚀 Visualize Results

In [0]:
# from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

vis = re_vis.display(result = light_result[0],
               relation_col = "relations",
               document_col = "document",
               exclude_relations = ["no_rel"],
               show_relations=True,
               return_html=True
               )

displayHTML(vis)

License fees revenue PROFIT_DECLINE decreased 40 PERCENTAGE %, or $ 0.5 million AMOUNT to $ 0.7 million AMOUNT for the year ended December 31, 2020 compared to $ 1.2 million AMOUNT for the year ended December 31, 2019. Services revenue PROFIT_INCREASE increased 4 PERCENTAGE %, or $ 1.1 million AMOUNT , to $ 25.6 million AMOUNT for the year ended December 31, 2020 from $ 24.5 million AMOUNT for the year ended December 31, 2019. Sales and marketing expenses EXPENSE_DECREASE decreased 20 PERCENTAGE %, or $ 1.5 million AMOUNT , to $ 6.0 million AMOUNT for the year ended December 31, 2020 from $ 7.5 million AMOUNT for the year ended December 31, 2019. <polyline fill="none" points="289.0,255.0 287.0612244897959,253.79607986468218 285.1224489795918,252.59312871337622 283.1836734693878,251.39211553009378 281.2448979591837,250.19400929884657 279.3061224489796,248.99977900364644 277.3673469387755,247.81039362850512 275.42857142857156,246.62682215743445 273.4897959183674,245.45003357444608 271.5510204081633,244.2809968635518 269.61224489795916,243.12068100876334 267.67346938775506,241.97005499409255 265.734693877551,240.8300878035512 263.795918367347,239.70174842115102 261.8571428571429,238.58600583090382 259.91836734693885,237.4838290168213 257.97959183673476,236.39618696291515 256.04081632653066,235.32404865319725 254.10204081632654,234.26838307167935 252.16326530612247,233.23015920237316 250.22448979591834,232.21034602929052 248.2857142857143,231.20991253644314 246.34693877551024,230.22982770784284 244.40816326530614,229.2710605275013 242.46938775510205,228.33457997943037 240.53061224489798,227.42135504764173 238.59183673469389,226.53235471614718 236.65306122448982,225.6685479689585 234.71428571428575,224.83090379008752 232.77551020408166,224.0203911635458 230.83673469387756,223.23797907334531 228.89795918367352,222.4846365034977 226.9591836734694,221.7613324380148 225.0204081632653,221.0690358609083 223.08163265306126,220.40871575619002 221.14285714285717,219.78134110787175 219.20408163265307,219.18788089996517 217.265306122449,218.6293041164821 215.3265306122449,218.10657974143427 213.38775510204084,217.6206767588335 211.44897959183675,217.1725641526915 209.51020408163268,216.76321090702004 207.57142857142858,216.3935860058309 205.63265306122452,216.06465843313583 203.69387755102042,215.77739717294665 201.75510204081633,215.53277120927507 199.8163265306123,215.33174952613282 197.87755102040816,215.1753011075317 195.9387755102041,215.06439493748354 194.0,215.0 194.0,215.0 192.06122448979593,214.98276228442228 190.1224489795918,215.0120358014092 188.18367346938774,215.08685156694915 186.24489795918367,215.20624059703016 184.3061224489796,215.36923390764048 182.36734693877548,215.57486251476848 180.42857142857147,215.82215743440238 178.48979591836735,216.11014968253025 176.55102040816325,216.43787027514048 174.61224489795916,216.8043502282212 172.6734693877551,217.20862055776075 170.73469387755097,217.64971227974735 168.79591836734693,218.1266564101692 166.85714285714286,218.63848396501453 164.9183673469388,219.18422596027168 162.97959183673467,219.7629134119287 161.0408163265306,220.37357733597398 159.10204081632654,221.01524874839563 157.16326530612244,221.68695866518203 155.22448979591832,222.38773810232126 153.28571428571425,223.1166180758017 151.3469387755102,223.87262960161155 149.4081632653061,224.654803695739 147.46938775510205,225.46217137417233 145.53061224489792,226.2937636528997 143.59183673469386,227.1486115479094 141.6530612244898,228.02574607518974 139.71428571428572,228.92419825072886 137.77551020408163,229.84299909051498 135.83673469387756,230.7811796105364 133.89795918367346,231.73777082678137 131.9591836734694,232.711803755238 130.0204081632653,233.7023094118947 128.0816326530612,234.70831881273955 126.14285714285715,235.72886297376093 124.20408163265307,236.76297291094693 122.26530612244898,237.80967964028594 120.3265306122449,238.86801417776604 118.38775510204081,239.9370075393756 116.44897959183675,241.01569074110